# Import and install packages

In [49]:
#import packages
import tabula
import pandas as pd
import time
import asyncio
from playwright.async_api import async_playwright
from bs4 import BeautifulSoup as bs
import plotly.express as px
import plotly.figure_factory as ff
import plotly as py
import plotly.graph_objects as go
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [ ]:
!pip install --upgrade plotly
!pip install --upgrade geopandas
!pip install --upgrade pyshp
!pip install --upgrade shapely

# Counts of Cancer per US State 2016-2020

In [52]:
#Cancer Occurence per US State

url ="https://acsjournals.onlinelibrary.wiley.com/doi/10.3322/caac.21763"


async with async_playwright() as p:
    browser = await p.chromium.launch(headless=False)
    page = await browser.new_page()
    await page.goto(url)
    time.sleep(6)
    html = await page.content()
    soup = bs(html, 'html.parser')
    #print(soup)

    #uses beautiful soup package to go through website and extract out table
    #appends data to list
    data = []
    m= soup.find(id="caac21763-tbl-0002", class_="article-table-content") 
    for d in m.find_all('div',{'class':"article-table-content-wrapper"}):
        table = m.find('table', attrs={'class':'table article-section__table'})
        table_body = table.find('tbody')
        rows = table_body.find_all('tr')
        for row in rows:
            cols = row.find_all('td')
            cols = [ele.text.strip() for ele in cols]
            data.append([ele for ele in cols if ele]) # Get rid of empty values
    


In [53]:
#converts list to dataframe
df = pd.DataFrame (data, columns = ['State', 'All sites', 'Female breast', 'Colon & rectum', 'Leukemia', 'Lung & bronchus', 'Melanoma of the skin', 'Non-Hodgkin lymphoma'
                                   ,'Prostate', 'Urinary bladder', 'Uterine cervix', 'Uterine corpus'])



In [54]:
#replaces value
df=df.replace('—b', '< 50')

#top 3 cancers in USA:
    #1. Breast
    #2. Prostate
    #3. Lung & Bronchus

#gets rid of last row
states = df.iloc[:-1 , :]
#replace char
states= states.replace(',','', regex=True)

#makes columns type int
c=0
for i in states:
    if(c!=0 and c!=10):
        states[i]=states[i].astype('Int64')
    c+=1
        

In [57]:
df.to_excel('unitedstates/Cancer in USA 2016-2020.xlsx')

## Population of US States 2020

In [59]:
#POPULATION OF US STATES 2020

url = "https://ballotpedia.org/United_States_census,_2020"



async with async_playwright() as p:
    browser = await p.chromium.launch(headless=False)
    page = await browser.new_page()
    await page.goto(url)
    time.sleep(6)
    html = await page.content()
    soup = bs(html, 'html.parser')
    #print(soup)

    #uses beautiful soup package to go through website and extract out table
    #appends data to list
    data = []
    
    for d in soup.find_all('div',{'class':"mw-parser-output"}):
        table = d.find('table', attrs={'class':'bptable collapsible sortable jquery-tablesorter'})
        table_body = table.find('tbody')
        rows = table_body.find_all('tr')
        for row in rows:
            cols = row.find_all('td')
            cols = [ele.text.strip() for ele in cols]
            data.append([ele for ele in cols if ele]) # Get rid of empty values

In [66]:
#converts list to dataframe
popstates2020 = pd.DataFrame (data, columns = ['State', 'Population','A','B','C'])
#delete columns
del popstates2020['A']
del popstates2020['B']
del popstates2020['C']
#replace char and convert the column of type int
popstates2020['Population'] = popstates2020['Population'].str.replace(',', '').astype(int)
#resets index and deletes column
popstates2020=popstates2020.reset_index()
del popstates2020['index']

## Choropleth Maps for all US States

In [67]:
#US codes w/ D.C
code=['AL', 'AK','AZ', 'AR', 'CA' ,'CO', 'CT','DC', 'DE', 'FL', 'GA', 'HI', 'ID', 'IL', 'IN', 'IA', 'KS', 
     'KY', 'LA', 'ME', 'MD', 'MA', 'MI', 'MN', 'MS', 'MO','MT', 'NE', 'NV', 'NH', 'NJ', 'NM', 'NY', 'NC', 'ND', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VT', 'VA', 'WA', 'WV', 'WI', 'WY']

In [68]:
#US codes w/o D.C
code2=['AL', 'AK','AZ', 'AR', 'CA' ,'CO', 'CT','DE', 'FL', 'GA', 'HI', 'ID', 'IL', 'IN', 'IA', 'KS', 
     'KY', 'LA', 'ME', 'MD', 'MA', 'MI', 'MN', 'MS', 'MO','MT', 'NE', 'NV', 'NH', 'NJ', 'NM', 'NY', 'NC', 'ND', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VT', 'VA', 'WA', 'WV', 'WI', 'WY']

### 2020 Population choropleth maps

In [76]:
#heatmap for 2020 population per state
fig = px.choropleth(popstates2020, locations=code2,
                    locationmode="USA-states", color='Population',color_continuous_scale="Viridis", scope="usa", title='United States Population')
#on click of state goes to heatmap for that state and its county
fig.update_layout(width=1000, height=600)
fig.write_html('unitedstates/United States Population.html')
fig.show()

### Cancer per US State choropleth map 

In [71]:
#heatmap for number of cancer per state
fig = px.choropleth(states, locations=code,
                    locationmode="USA-states", color='All sites',color_continuous_scale="Viridis", scope="usa", title='Cancer per State from 2016-2020')
#on click of state goes to heatmap for that state and its county
fig.show()
fig.write_html('unitedstates/Cancer per State from 2016-2020.html')

In [13]:
#https://cancerstatisticscenter.cancer.org/#!/state/New%20York

# Annual Cancer per County in NY 

In [62]:
#read in pdf of annual cancer cases per county in nys
df=tabula.read_pdf("newyork/volume1.pdf", pages="all")  
#pages of pdfs



In [63]:
#extracts data for given borough
def extract(i,name):
    nys=df[i].iloc[:, [0, 1]]
    nys=nys[3:]
    nys.columns = nys.iloc[0]
    nys = nys[1:]
    nys.rename(columns={nys.columns[1]: name},inplace=True)
    site=nys.iloc[:, [0]]
    nys=nys[name].str.split(' ', expand=True)
    nys['Site of Cancer']=site
    del nys[1]
    del nys[2]
    nys=nys.rename(columns={0:name})
    first_column = nys.pop('Site of Cancer')
    nys.insert(0, 'Site of Cancer', first_column)
    return nys
        
        
            

In [64]:
#list of county names
county_name=['Bronx','Brooklyn','Manhattan','Queens','Staten Island','Albany','Allegany','Broome','Cattaraugus','Cayuga','Chautauqua','Chemung','Chenago','Clinton','Columbia','Cortland','Delaware','Dutchess','Erie','Essex','Franklin','Fulton','Genesee','Greene','Hamilton','Herkimer','Jefferson','Lewis','Livingston','Madison','Monroe','Montgomery','Nassau','Niagra','Oneida','Onondaga','Ontario','Orange','Orleans','Oswego','Otsego','Putnam','Rensselaer','Rockland','St. Lawrence','Saratoga','Schenectady','Schoharie','Schuyler','Seneca','Steuben','Suffolk','Sullivan','Tioga','Tompkins','Ulster','Warren','Washington','Wayne','Westchester','Wyoming','Yates']
#merge dfs together
i=3
j=0
while(i<65):
    c_county=extract(i,county_name[j])
    if(j>0):
        county=pd.merge(county, c_county, on=['Site of Cancer'])
    else:
        county=c_county
    i+=1
    j+=1

In [65]:
#Tranposes df (changes rows to columns)
county_NY=county.T
county_NY.columns = county_NY.iloc[0]
county_NY = county_NY[1:]

#Reset index and rename columns
county_NY=county_NY.reset_index()
county_NY=county_NY.rename(columns={'index':'County'})

In [67]:
#converts data to float
county_NY=county_NY.set_index('County')
for i in county_NY.columns:
    county_NY[i] = county_NY[i].str.replace(',', '')
    county_NY[i]=county_NY[i].astype(float)
    
county_NY=county_NY.reset_index()

In [70]:
#POPULATION DATA

#uses beautiful soup package to go through website and extract out table
#appends data to list
url='https://www.health.ny.gov/statistics/cancer/registry/appendix/countypop.htm'
data=[]
async with async_playwright() as p:
    browser = await p.chromium.launch(headless=False)
    page = await browser.new_page()
    await page.goto(url)
    time.sleep(6)
    html = await page.content()
    soup = bs(html, 'html.parser')
    #print(soup)
    for d in soup.find_all('div',{'id':"content"}):
        table = d.find('table', attrs={'class':'light_table right'})
        table_body = table.find('tbody')
        rows = table_body.find_all('tr')
        for row in rows:
            cols = row.find_all('td')
            cols = [ele.text.strip() for ele in cols]
            data.append([ele for ele in cols if ele]) # Get rid of empty values


In [71]:
#slice list
d1=data[3:]
#converts list to dataframe
pop= pd.DataFrame (d1, columns = ['Males','Female','Total Population'])
county_name=['Bronx','Brooklyn','Manhattan','Queens','Staten Island','Albany','Allegany','Broome','Cattaraugus','Cayuga','Chautauqua','Chemung','Chenago','Clinton','Columbia','Cortland','Delaware','Dutchess','Erie','Essex','Franklin','Fulton','Genesee','Greene','Hamilton','Herkimer','Jefferson','Lewis','Livingston','Madison','Monroe','Montgomery','Nassau','Niagra','Oneida','Onondaga','Ontario','Orange','Orleans','Oswego','Otsego','Putnam','Rensselaer','Rockland','St. Lawrence','Saratoga','Schenectady','Schoharie','Schuyler','Seneca','Steuben','Suffolk','Sullivan','Tioga','Tompkins','Ulster','Warren','Washington','Wayne','Westchester','Wyoming','Yates']
#assign list to column in dataframe
pop['Counties']=county_name

In [72]:
#replace columns chars w another char and convert to type int
pop['Males'] = pop['Males'].str.replace(',', '').astype(int)
pop['Female'] = pop['Female'].str.replace(',', '').astype(int)
pop['Total Population'] = pop['Total Population'].str.replace(',', '').astype(int)
#pop['Males']=pop['Males'].astype(float)

#sort values in column by descending
pop=pop.sort_values('Total Population', ascending=False)
#sort values in column by ascending
pop=pop.sort_values('Counties')

# Choropleth Maps

## Cancer per County NY

In [68]:
#Fips of NY counties
NY_fips=['36001','36003','36005','36047','36007','36009','36011','36013','36015','36017','36019','36021','36023','36025','36027','36029','36031','36033',
        '36035','36037','36039','36041','36043','36045','36049','36051','36053','36061','36055','36057','36059','36063','36065','36067',
        '36069','36071','36073','36075','36077','36079','36081','36083','36087','36091','36093','36095','36097','36099','36089','36085','36101',
        '36103','36105','36107','36109','36111','36113','36115','36117','36119','36121','36123']

In [69]:
#heatmap for 2016-2020 cancer per county in NY


county_NY=county_NY.sort_values('County')
#grabs county names
county_val=county_NY['County'].tolist()
values=county_NY['All Invasive Malignant Tumors'].tolist()
endpts = list(np.mgrid[min(values):max(values):6j])
fig =ff.create_choropleth(fips=NY_fips,values=values,scope=["New York"],binning_endpoints=endpts, show_state_data=True,county_outline={'color': 'rgb(255,255,255)', 'width': 0.5},round_legend_values=True,title='Count of Cancer per County in New York from 2016-2020')
#on click of state goes to heatmap for that state and its county
py.offline.plot(fig,
                filename='choropleth_newyork',
                include_plotlyjs='https://cdn.plot.ly/plotly-1.42.3.min.js')

'choropleth_newyork.html'

## Population per County NY

In [73]:
values=pop['Total Population'].tolist()
endpts = list(np.mgrid[min(values):max(values):6j])
#heatmap for 2016-2020 population per county in NY
fig =ff.create_choropleth(fips=NY_fips,values=values,scope=["New York"],binning_endpoints=endpts, show_state_data=True,county_outline={'color': 'rgb(255,255,255)', 'width': 0.5},round_legend_values=True,title='Count of Pop per County in New York from 2016-2020')
#on click of state goes to heatmap for that state and its county
py.offline.plot(fig,
                filename='choropleth_pop_newyork',
                include_plotlyjs='https://cdn.plot.ly/plotly-1.42.3.min.js')

'choropleth_pop_newyork.html'

# Most common cancer for all 62 counties : Lung and Bronchus

In [74]:
max_county_ny=county_NY['County']
county_NY_cluster=county_NY.copy()
del county_NY_cluster['County']
del county_NY_cluster['All Invasive Malignant Tumors']

In [75]:
#grabs the most prevalent cancer in all the counties
max_county_ny['Area Site']=county_NY_cluster.idxmax(axis=1)
max_ny_num=max_county_ny.groupby(max_county_ny['Area Site']).count()

# County with the most type of each cancer

In [79]:
#makes each column float values

county=county.set_index('Site of Cancer')

for i in county.columns:
    county[i] = county[i].str.replace(',', '')
    county[i]=county[i].astype(float)

county=county.reset_index()

In [80]:
#drops indexes
county=county.drop([10,11,12,13,14])
county=county.set_index('Site of Cancer')

In [81]:
#grabs top county that has the most accounts of a certain type of cancer

def top(c):
    # Define the number of top values to return
    n = 1
    i=[]
    t=[]
    # Iterate over each row of the DataFrame
    for index, row in c.iterrows():
        # Get the top-n highest values in the row
        top_n = row.nlargest(n)
        i.append(index)
        t.append(list(top_n.index))


        # Print the names of the top-n highest-value columns
        #print(f'Top column in row {index}: {list(top_n.index)}')
        #print()
    return i,t

    

In [82]:
#call function from above and put it in df

top_cancer_county=pd.DataFrame(columns=['Site','County'])
top_cancer_county['Site'],top_cancer_county['Site of Cancer']=top(county)
del top_cancer_county['County']
top_cancer_county=top_cancer_county.rename(columns={'Site of Cancer':'County'})

#converts list to string
top_cancer_county['County'] = [','.join(map(str, l)) for l in top_cancer_county['County']]
top_cancer_county=top_cancer_county[1:]

In [86]:
top_cancer_county

,Site,County
1,Oral cavity and pharynx,Brooklyn
2,Esophagus,Queens
3,Colorectal,Brooklyn
4,Colon excluding rectum,Brooklyn
5,Rectum & rectosigmoid,Brooklyn
6,Liver / intrahepatic bile duct,Brooklyn
7,Pancreas,Brooklyn
8,Lung and bronchus,Suffolk
9,Melanoma of the skin,Suffolk
10,Urinary bladder (incl. in situ),Suffolk


In [87]:
#group by to get counts per county
top_county_per_site=top_cancer_county.groupby(top_cancer_county['County']).count()
print(top_county_per_site)

          Site
County        
Brooklyn    12
Queens       1
Suffolk      4


In [88]:
top_cancer_county.loc[top_cancer_county['County']=='Suffolk']

,Site,County
8,Lung and bronchus,Suffolk
9,Melanoma of the skin,Suffolk
10,Urinary bladder (incl. in situ),Suffolk
17,Leukemias,Suffolk


# Florida Extraction

In [2]:
fl1=tabula.read_pdf("florida/Florida 2016 cases.pdf", pages="all")  
fl2=tabula.read_pdf("florida/Florida 2017 cases.pdf", pages="all")  
fl3=tabula.read_pdf("florida/Florida 2018 cases.pdf", pages="all")  
fl4=tabula.read_pdf("florida/Florida 2019 cases.pdf", pages="all")  
fl5=tabula.read_pdf("florida/Florida 2020 cases.pdf", pages="all")  

In [3]:
#grabs data from pdf and put in df
def florida_extract(fl):
    florida=fl[0]
    florida.columns = florida.iloc[0]
    florida = florida[1:]
    florida.rename(columns={florida.columns[0]: 'County'},inplace=True)
    florida=florida[1:]
    florida=florida.replace('^','0')
    #florida['Cancers'] = florida['Cancers'].str.replace(',', '').astype(int)
    return florida

In [4]:
#merges dfs
fl1=florida_extract(fl1)
fl2=florida_extract(fl2)
fl3=florida_extract(fl3)
fl4=florida_extract(fl4)
fl5=florida_extract(fl5)

fl1=fl1.rename(columns={'All Cancers':'Cancers'})

florida=pd.DataFrame()

k=0
for i in fl1.columns:
    if(k!=0):
        fl1[i] = fl1[i].str.replace(',', '').astype(int)
        fl2[i] = fl2[i].str.replace(',', '').astype(int)
        fl3[i] = fl3[i].str.replace(',', '').astype(int)
        fl4[i] = fl4[i].str.replace(',', '').astype(int)
        fl5[i] = fl5[i].str.replace(',', '').astype(int)
        florida[i]=fl1[i]+fl2[i]+fl3[i]+fl4[i]+fl5[i]
    else:
        florida[i]=fl1[i]
    k+=1
    

In [8]:
florida.to_csv('Florida Cancer 2016-2020.csv')

# Choropleth Map of Cancer per Florida County

In [11]:
fips=['12001', '12003', '12005', '12007', '12009', '12011', '12013', '12015', '12017', '12019', '12021', '12023','12027', '12029', '12031','12033','12035','12037','12039','12041','12043','12045','12047','12049','12051','12053','12055',
      '12057','12059','12061','12063','12065','12067','12069','12071','12073','12075','12077','12079','12081','12083',
      '12085','12086','12087','12089','12091','12093','12095','12097','12099','12101','12103','12105','12107','12109','12111',
      '12113','12115','12117','12119','12121','12123','12125','12127','12129','12131','12133']

In [15]:
county_fl=florida['County'].tolist()
values=florida['Cancers'].tolist()
endpts = list(np.mgrid[min(values):max(values):6j])
fig =ff.create_choropleth(fips=fips,values=values,scope=["Florida"],binning_endpoints=endpts, show_state_data=True,county_outline={'color': 'rgb(255,255,255)', 'width': 0.5},round_legend_values=True,title='Count of Cancer per County in Florida from 2016-2020')
#on click of state goes to heatmap for that state and its county
py.offline.plot(fig,
                filename='choropleth_florida',
                include_plotlyjs='https://cdn.plot.ly/plotly-1.42.3.min.js')

In [19]:
cancer_fl=florida.sort_values('Cancers', ascending=False)
cancer_fl.to_excel("florida/Cancer per County.xlsx")
cancer_fl.to_html("florida/Cancer per County.html")

# Showing how many counties cancers are most prevalent in

In [32]:
#create df and fill column with existing data from another df
max_florida=pd.DataFrame(columns=['County'])
max_florida['County']=florida['County']

In [35]:
#creates copy of df
florida_cluster=florida.copy()
del florida_cluster['County']
del florida_cluster['Cancers']

#grabs each cancer site and counts how many counties it's the most prevalent
max_florida['Area Site']=florida_cluster.idxmax(axis=1)
max_florida_num=max_florida.groupby(max_florida['Area Site']).count()

In [40]:
max_florida_num.sort_values('County', ascending=False)

,County
Area Site,
Bronchus,50
Breast,15
Melanoma,1
Prostate,1


In [41]:
max_florida

,County,Area Site
2,Alachua,Breast
3,Baker,Bronchus
4,Bay,Bronchus
5,Bradford,Bronchus
6,Brevard,Bronchus
...,...,...
64,Union,Prostate
65,Volusia,Bronchus
66,Wakulla,Bronchus
67,Walton,Bronchus
